In [2]:
import os
from groq import Groq

In [3]:
groq_api_key = "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"
client = Groq(api_key=groq_api_key)

In [32]:
def summarize_text(text, max_words, retries=3):
    prompt = f"Summarize the following text in approximately {max_words} words. It's crucial to keep the summary as close to {max_words} words as possible:\n\n{text}\n\nSummary:"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": f"You are a helpful assistant that provides concise summaries. Always aim to keep your summaries as close to the requested word count as possible."},
            {"role": "user", "content": prompt}
        ],
        model="mixtral-8x7b-32768",
        temperature=0.7,
        top_p=1,
        max_tokens=max_words * 2
    )

    return chat_completion.choices[0].message.content

In [5]:
import PyPDF2
import re

In [6]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [33]:
def extract_pdf_content(base_path, pdf_file):
    full_path = os.path.join(base_path, pdf_file)
    with open(full_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return clean_text(text)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [52]:
def splitter_chunks(text, filename):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    langchain_doc = Document(page_content=text, metadata={"source": filename})
    return text_splitter.split_documents([langchain_doc])

In [64]:
base_path = 'F:/Super AI SS4/AI-builder/Large-Language-Models-(LLMs)/try-it/Text-Summarization/Dataset/'
pdf_files = ['document1.pdf', 'document2.pdf', 'document3.pdf']

In [65]:
first_content = extract_pdf_content(base_path, pdf_files[0])
first_content

"Climate change is one of the most pressing global challenges of our time. It refers to long-term shifts in temperature and weather patterns across the Earth. While these changes can occur naturally, the rapid warming we've seen over the past century is primarily due to human activities. The main driver of current climate change is the burning of fossil fuels like coal, oil, and natural gas. These activities release greenhouse gases, primarily carbon dioxide (CO2), into the atmosphere. Greenhouse gases trap heat from the sun, causing the Earth's average temperature to rise – a phenomenon known as global warming. The evidence for climate change is unequivocal. Global temperature records show that the Earth has warmed by approximately 1°C since pre -industrial times. This may seem small, but it represents a significant change in the Earth's energy balance and has far-reaching consequences. One of the most visible effects of climate change is the melting of polar ice and glaciers. The Arc

In [66]:
all_chunks = []

for pdf_file in pdf_files:
    print(f"Processing {pdf_file}...")
    content = extract_pdf_content(base_path, pdf_file)
    chunks = splitter_chunks(content, pdf_file)
    all_chunks.extend(chunks)

print(f"Total number of chunks across all documents: {len(all_chunks)}")

Processing document1.pdf...
Processing document2.pdf...
Processing document3.pdf...
Total number of chunks across all documents: 18


In [67]:
all_chunks

[Document(page_content="Climate change is one of the most pressing global challenges of our time. It refers to long-term shifts in temperature and weather patterns across the Earth. While these changes can occur naturally, the rapid warming we've seen over the past century is primarily due to human activities. The main driver of current climate change is the burning of fossil fuels like coal, oil, and natural gas. These activities release greenhouse gases, primarily carbon dioxide (CO2), into the atmosphere. Greenhouse gases trap heat from the sun, causing the Earth's average temperature to rise – a phenomenon known as global warming. The evidence for climate change is unequivocal. Global temperature records show that the Earth has warmed by approximately 1°C since pre -industrial times. This may seem small, but it represents a significant change in the Earth's energy balance and has far-reaching consequences. One of the most visible effects of climate change is the melting of polar ic

In [ ]:
'''
documents = []

for i, pdf_file in enumerate(pdf_files, 1):
    content = extract_pdf_content(base_path, pdf_file)
    document = {
        'id': i,
        'filename': pdf_file,
        'content': content,
        'length': len(content)
    }
    documents.append(document)

for doc in documents:
    print(f"Document ID: {doc['id']}")
    print(f"Filename: {doc['filename']}")
    print(f"Content length: {doc['length']} characters")
    print(f"Preview: {doc['content'][:100]}...")
    print("\n" + "="*50 + "\n")
'''

In [68]:
import time

In [69]:
chunk_summaries = []

for i, chunk in enumerate(all_chunks, 1):
    print(f"Summarizing chunk {i}/{len(all_chunks)}...")
    summary = summarize_text(chunk.page_content, 100)
    if summary:
        chunk_summaries.append(summary)
    time.sleep(1)

Summarizing chunk 1/18...
Summarizing chunk 2/18...
Summarizing chunk 3/18...
Summarizing chunk 4/18...
Summarizing chunk 5/18...
Summarizing chunk 6/18...
Summarizing chunk 7/18...
Summarizing chunk 8/18...
Summarizing chunk 9/18...
Summarizing chunk 10/18...
Summarizing chunk 11/18...
Summarizing chunk 12/18...
Summarizing chunk 13/18...
Summarizing chunk 14/18...
Summarizing chunk 15/18...
Summarizing chunk 16/18...
Summarizing chunk 17/18...
Summarizing chunk 18/18...


In [70]:
chunk_summaries

["Climate change, a pressing global issue, involves long-term shifts in temperature and weather patterns, largely caused by human activities since the 1800s. The burning of fossil fuels releases greenhouse gases, particularly carbon dioxide, trapping heat from the sun and leading to global warming. Evidence shows a 1°C rise in Earth's average temperature since industrial times, causing polar ice and glacier melting. This summary is approximately 100 words, providing a concise overview of the text.",
 'Climate change is causing the polar ice and glaciers to melt, resulting in faster warming in the Arctic and contributing to sea level rises worldwide. Coastal communities face floods and erosion due to expanding warmer water and added meltwater. Low-lying island nations are particularly at risk. Precipitation patterns are changing, leading to increased droughts and heavy rainfall in various regions, impacting agriculture, water resources, and human settlements. Extreme weather events, suc

In [73]:
combined_summaries = "\n\n".join(chunk_summaries)
final_summary = summarize_text(combined_summaries, 250)

print(final_summary)

Climate change, largely caused by human activities since the 1800s, involves long-term shifts in temperature and weather patterns due to increased greenhouse gas emissions, primarily carbon dioxide. These shifts result in various impacts, including:

1. Rising global temperatures: A 1°C increase since industrial times has caused polar ice and glacier melting, contributing to sea level rises and threatening coastal communities with floods and erosion.
2. Extreme weather events: Climate change intensifies weather events, leading to longer, more severe heatwaves, intense hurricanes, and expansive wildfires, which affect ecosystems, biodiversity, and human societies.
3. Impact on ecosystems and biodiversity: As temperatures shift, plant and animal species must migrate or risk extinction. Ocean acidification threatens marine life, particularly coral reefs and shellfish.
4. Economic consequences: Climate change has far-reaching economic impacts, including damages from extreme weather events,